<center>

<span style='color:yellow'>
<center><h1> Task-01</h1> </center>
</span>
<h2> Implement a linear regression model to predict the prices of houses based on their square footage and the number of bedrooms and bathrooms. </h2>



Dataset : - https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

</center>


<center> 


# -----------------------------------------------------------------------------------

</center>

<center>

# Loading Libreries


</center>

In [122]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score



<center>

# Importing Data


 </center>

In [100]:
df = pd.read_csv("Data/train.csv")

In [101]:
df.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1213,1214,80,RL,NaN,10246,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,145000
380,381,50,RL,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,127000
626,627,20,RL,NaN,12342,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,GdWo,Shed,600,8,2007,WD,Normal,139900
1329,1330,60,RL,63.0,9084,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,176500
660,661,60,RL,NaN,12384,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,11,2007,WD,Normal,197900




<center>


### Using columns containing Square Footing and number of bedrooms and bathrooms

 </center>

In [102]:
with open('Data/column.txt', 'r') as file:
    columns = file.read().strip().split(',')

In [103]:
columns

['PoolArea',
 'ScreenPorch',
 '3SsnPorch',
 'EnclosedPorch',
 'OpenPorchSF',
 'WoodDeckSF',
 'GarageArea',
 'GrLivArea',
 'LowQualFinSF',
 'BsmtFinSF1',
 '2ndFlrSF',
 '1stFlrSF',
 'TotalBsmtSF',
 'BsmtUnfSF',
 'BsmtFinSF2',
 'MasVnrArea',
 'LotArea',
 'FullBath',
 'BsmtHalfBath',
 'BsmtFullBath',
 'BedroomAbvGr',
 'SalePrice']

In [104]:
df_filtered = df[columns]
print(df_filtered.info())
df_filtered

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PoolArea       1460 non-null   int64  
 1   ScreenPorch    1460 non-null   int64  
 2   3SsnPorch      1460 non-null   int64  
 3   EnclosedPorch  1460 non-null   int64  
 4   OpenPorchSF    1460 non-null   int64  
 5   WoodDeckSF     1460 non-null   int64  
 6   GarageArea     1460 non-null   int64  
 7   GrLivArea      1460 non-null   int64  
 8   LowQualFinSF   1460 non-null   int64  
 9   BsmtFinSF1     1460 non-null   int64  
 10  2ndFlrSF       1460 non-null   int64  
 11  1stFlrSF       1460 non-null   int64  
 12  TotalBsmtSF    1460 non-null   int64  
 13  BsmtUnfSF      1460 non-null   int64  
 14  BsmtFinSF2     1460 non-null   int64  
 15  MasVnrArea     1452 non-null   float64
 16  LotArea        1460 non-null   int64  
 17  FullBath       1460 non-null   int64  
 18  BsmtHalf

,PoolArea,ScreenPorch,3SsnPorch,EnclosedPorch,OpenPorchSF,WoodDeckSF,GarageArea,GrLivArea,LowQualFinSF,BsmtFinSF1,...,TotalBsmtSF,BsmtUnfSF,BsmtFinSF2,MasVnrArea,LotArea,FullBath,BsmtHalfBath,BsmtFullBath,BedroomAbvGr,SalePrice
0,0,0,0,0,61,0,548,1710,0,706,...,856,150,0,196.0,8450,2,0,1,3,208500
1,0,0,0,0,0,298,460,1262,0,978,...,1262,284,0,0.0,9600,2,1,0,3,181500
2,0,0,0,0,42,0,608,1786,0,486,...,920,434,0,162.0,11250,2,0,1,3,223500
3,0,0,0,272,35,0,642,1717,0,216,...,756,540,0,0.0,9550,1,0,1,3,140000
4,0,0,0,0,84,192,836,2198,0,655,...,1145,490,0,350.0,14260,2,0,1,4,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,0,40,0,460,1647,0,0,...,953,953,0,0.0,7917,2,0,0,3,175000
1456,0,0,0,0,0,349,500,2073,0,790,...,1542,589,163,119.0,13175,2,0,1,3,210000
1457,0,0,0,0,60,0,252,2340,0,275,...,1152,877,0,0.0,9042,2,0,0,4,266500
1458,0,0,0,112,0,366,240,1078,0,49,...,1078,0,1029,0.0,9717,1,0,1,2,142125


In [105]:
missing_values = df_filtered.isnull().sum()
print(f"Missing values after imputation:\n{missing_values}")


Missing values after imputation:
PoolArea         0
ScreenPorch      0
3SsnPorch        0
EnclosedPorch    0
OpenPorchSF      0
WoodDeckSF       0
GarageArea       0
GrLivArea        0
LowQualFinSF     0
BsmtFinSF1       0
2ndFlrSF         0
1stFlrSF         0
TotalBsmtSF      0
BsmtUnfSF        0
BsmtFinSF2       0
MasVnrArea       8
LotArea          0
FullBath         0
BsmtHalfBath     0
BsmtFullBath     0
BedroomAbvGr     0
SalePrice        0
dtype: int64


In [108]:
missing_values = df_filtered.isnull().sum()
print(f"Missing values after imputation:\n{missing_values}")


Missing values after imputation:
PoolArea         0
ScreenPorch      0
3SsnPorch        0
EnclosedPorch    0
OpenPorchSF      0
WoodDeckSF       0
GarageArea       0
GrLivArea        0
LowQualFinSF     0
BsmtFinSF1       0
2ndFlrSF         0
1stFlrSF         0
TotalBsmtSF      0
BsmtUnfSF        0
BsmtFinSF2       0
MasVnrArea       0
LotArea          0
FullBath         0
BsmtHalfBath     0
BsmtFullBath     0
BedroomAbvGr     0
SalePrice        0
dtype: int64


In [109]:
# data=df_imputed.values
data = df_filtered.to_numpy()
data

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        3.00000e+00, 2.08500e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        3.00000e+00, 1.81500e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        3.00000e+00, 2.23500e+05],
       ...,
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        4.00000e+00, 2.66500e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        2.00000e+00, 1.42125e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        3.00000e+00, 1.47500e+05]])

<center>


# Test and Train Split

 </center>

In [110]:
X,y = data[:,:-1],data[:,-1]

In [111]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1168, 21)
(1168,)
(292, 21)
(292,)


<center>

# Linear Regression


 </center>

In [113]:
model = LinearRegression()


In [114]:
model.fit(X_train,y_train)

LinearRegression()

In [115]:
y_pred = model.predict(X_test)

In [116]:
y_pred.shape

(292,)

In [117]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 1735691334.6501758
R^2 Score: 0.7737134917386139


<center>

# Cross Validation


 </center>

In [120]:
cv_results = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')


cv_mse = -cv_results  
cv_rmse = (cv_mse.mean())**0.5 
cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2').mean()

print(f"Cross-Validated Mean Squared Error: {cv_mse.mean()}")
print(f"Cross-Validated Root Mean Squared Error: {cv_rmse}")
print(f"Cross-Validated R^2 Score: {cv_r2}")

Cross-Validated Mean Squared Error: 1948007840.3045914
Cross-Validated Root Mean Squared Error: 44136.24180086691
Cross-Validated R^2 Score: 0.6920042603796559


<center>

# Polynomial Regression


 </center>

In [129]:

degree = 2
model = make_pipeline(PolynomialFeatures(degree), StandardScaler(), LinearRegression())

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")


Mean Squared Error (MSE): 2.117700955491694e+38
